### Imports

In [94]:
import pandas as pd
from neo4j import GraphDatabase


### Neo4j Credentials

In [95]:
NEO4J_URI = "neo4j+ssc://b931b8dd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "mW-0f4ShNF394EffXlczpH-5onIZuEsXRFatduQQN5I" 
NEO4J_DATABASE = "neo4j"

### Data Loading & Cleaning

In [150]:
import pandas as pd
import numpy as np

def load_and_clean_data(filepath):
    df = pd.read_csv(filepath)

    # 1. Date Cleaning
    date_cols = ["DateofHire", "DateofTermination", "DOB", "LastPerformanceReview_Date"]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    
    # 2.---- Text Cleanup ----
    df.fillna("", inplace=True)
    
    # Force string type and strip whitespace from ALL string columns
    string_cols = ["Employee_Name", "Position", "Department", "RecruitmentSource", "State", "Sex", "RaceDesc", "MaritalDesc", "CitizenDesc", "TermReason", "EmploymentStatus"]
    
    for col in string_cols:
        if col in df.columns:
            # excessive whitespace removal
            df[col] = df[col].astype(str).str.strip()

    # 3. String/ID Cleaning
    df["EmpID"] = df["EmpID"].astype(str)
    df.fillna("", inplace=True)

    def clean_id(x):
        try:
            return int(float(x)) if x != "" else None
        except:
            return None

    records = []
    for _, row in df.iterrows():
        
        # Logic: Status Normalization
        has_term_date = pd.notna(row["DateofTermination"])
        raw_status = str(row["EmploymentStatus"])
        final_status = "Terminated" if (has_term_date or "Terminated" in raw_status) else "Active"

        # Logic: Termination Reason
        raw_reason = str(row["TermReason"])
        term_reason = raw_reason if (raw_reason and "N/A" not in raw_reason and "StillEmployed" not in raw_reason) else None

        record = {
            # --- Primary Identity ---
            "emp_id": row["EmpID"],
            "name": row["Employee_Name"],
            
            # --- IDs (All 36 cols requirement) ---
            "married_id": clean_id(row["MarriedID"]),
            "marital_status_id": clean_id(row["MaritalStatusID"]),
            "gender_id": clean_id(row["GenderID"]),
            "emp_status_id": clean_id(row["EmpStatusID"]),
            "dept_id": clean_id(row["DeptID"]),
            "perf_score_id": clean_id(row["PerfScoreID"]),
            "diversity_id": clean_id(row["FromDiversityJobFairID"]),
            "position_id": clean_id(row["PositionID"]),
            "manager_id": clean_id(row["ManagerID"]),
            "termd": clean_id(row["Termd"]), # Boolean-like integer
            
            # --- Demographics ---
            "dob": row["DOB"].date().isoformat() if pd.notna(row["DOB"]) else None,
            "sex": row["Sex"],
            "race": row["RaceDesc"],
            "marital_status": row["MaritalDesc"],
            "citizen_desc": row["CitizenDesc"],        # Previously missing
            "hispanic_latino": row["HispanicLatino"],  # Previously missing
            
            # --- Job Details ---
            "salary": float(row["Salary"]) if row["Salary"] else 0.0,
            "position_title": row["Position"],
            "department": row["Department"],
            "manager_name": row["ManagerName"],
            "recruitment_source": row["RecruitmentSource"],
            "employment_status": final_status,         # Cleaned
            "raw_employment_status": row["EmploymentStatus"], # Original (for completeness)
            
            # --- Dates ---
            "hire_date": row["DateofHire"].date().isoformat() if pd.notna(row["DateofHire"]) else None,
            "term_date": row["DateofTermination"].date().isoformat() if pd.notna(row["DateofTermination"]) else None,
            "last_review_date": row["LastPerformanceReview_Date"].date().isoformat() if pd.notna(row["LastPerformanceReview_Date"]) else None,
            
            # --- Performance & Metrics ---
            "performance_score": row["PerformanceScore"],
            "engagement_score": float(row["EngagementSurvey"]) if row["EngagementSurvey"] else 0.0,
            "satisfaction_score": int(row["EmpSatisfaction"]) if row["EmpSatisfaction"] else 0,
            "project_count": int(row["SpecialProjectsCount"]) if row["SpecialProjectsCount"] else 0,
            "absences": int(row["Absences"]) if row["Absences"] else 0,
            "days_late_last_30": int(row["DaysLateLast30"]) if row["DaysLateLast30"] else 0, # Previously missing
            
            # --- Location & Termination ---
            "state": row["State"],
            "zip": str(row["Zip"]),
            "term_reason": term_reason,                # Cleaned
            "raw_term_reason": row["TermReason"]       # Original (for completeness)
        }
        records.append(record)

    return records

### Constraints

In [151]:
def create_constraints(driver):
    queries = [
        # Entity Uniqueness
        "CREATE CONSTRAINT IF NOT EXISTS FOR (e:Employee) REQUIRE e.emp_id IS UNIQUE",
        
        # Category Uniqueness (Crucial for connecting shared nodes)
        "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department) REQUIRE d.name IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Position) REQUIRE p.title IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Source) REQUIRE s.name IS UNIQUE",
        
        # New Insights Nodes
        "CREATE CONSTRAINT IF NOT EXISTS FOR (l:Location) REQUIRE l.state IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (pr:PerformanceRating) REQUIRE pr.rating IS UNIQUE",
        "CREATE CONSTRAINT IF NOT EXISTS FOR (r:Reason) REQUIRE r.desc IS UNIQUE"
    ]

    with driver.session(database=NEO4J_DATABASE) as session:
        for q in queries:
            session.run(q)

    print("✅ Constraints created/verified.")

### KG Ingestion

In [152]:
def ingest_batch(driver, data):
    query = """
    UNWIND $rows AS row

    // 1. Create Lookup/Category Nodes
    MERGE (d:Department {name: row.department})
    MERGE (pos:Position {title: row.position_title})
    SET pos.position_id = row.position_id
    MERGE (src:Source {name: row.recruitment_source})
    MERGE (loc:Location {state: row.state})
    MERGE (perf:PerformanceRating {rating: row.performance_score})

    // 2. Create Employee (ALL Attributes Mapped)
    MERGE (e:Employee {emp_id: row.emp_id})
    SET
        e.name = row.name,
        e.salary = row.salary,
        
        // Dates
        e.dob = date(row.dob),
        e.hire_date = date(row.hire_date),        
        e.last_review = date(row.last_review_date),
        
        // Demographics
        e.sex = row.sex,
        e.race = row.race,
        e.marital_status = row.marital_status,
        e.citizen_desc = row.citizen_desc,        
        e.hispanic_latino = row.hispanic_latino,  
        
        // Status
        e.employment_status = row.employment_status,
        e.raw_employment_status = row.raw_employment_status,
        e.termd = row.termd,
        
        // Metrics
        e.engagement_score = row.engagement_score,
        e.satisfaction_score = row.satisfaction_score,
        e.project_count = row.project_count,
        e.absences = row.absences,
        e.days_late_last_30 = row.days_late_last_30, 
        
        // IDs (Meta-data)
        e.married_id = row.married_id,
        e.marital_status_id = row.marital_status_id,
        e.gender_id = row.gender_id,
        e.emp_status_id = row.emp_status_id,
        e.dept_id = row.dept_id,
        e.perf_score_id = row.perf_score_id,
        e.diversity_id = row.diversity_id

    // 3. Relationships
    MERGE (e)-[:WORKS_IN]->(d)
    MERGE (e)-[:RECRUITED_FROM]->(src)
    MERGE (e)-[:LOCATED_IN {zip: row.zip}]->(loc)
    MERGE (e)-[:HAS_PERFORMANCE_RATING]->(perf)

    // Position History
    MERGE (e)-[r:HELD_POSITION]->(pos)
    SET r.from = date(row.hire_date)
    FOREACH (_ IN CASE WHEN row.term_date IS NOT NULL THEN [1] ELSE [] END |
        SET r.to = date(row.term_date),
            e.term_date = date(row.term_date) // Set on node too
    )

    // Termination Reason
    FOREACH (_ IN CASE WHEN row.term_reason IS NOT NULL THEN [1] ELSE [] END |
        MERGE (reason:Reason {desc: row.term_reason})
        MERGE (e)-[:TERMINATED_DUE_TO]->(reason)
    )

    // Manager Hierarchy
    WITH e, row
    WHERE row.manager_id IS NOT NULL 
    MERGE (m:Employee {emp_id: toString(row.manager_id)})
    ON CREATE SET m.name = row.manager_name 
    MERGE (e)-[:REPORTS_TO]->(m)
    """

    with driver.session(database=NEO4J_DATABASE) as session:
        session.run(query, rows=data)
    
    print(f"✅ Ingested {len(data)} records with full attribute mapping.")

### Main Execution Cell

In [153]:
if __name__ == "__main__":
    driver = GraphDatabase.driver(
        NEO4J_URI,
        auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    try:
        print("Loading and cleaning data...")
        data = load_and_clean_data("HRDataset_v14.csv")

        print("Creating constraints...")
        create_constraints(driver)

        print("Ingesting Knowledge Graph...")
        ingest_batch(driver, data)

        print("Knowledge Graph construction completed successfully.")

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        driver.close()



Loading and cleaning data...


C:\Users\admin\AppData\Local\Temp\ipykernel_6372\1395517079.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\admin\AppData\Local\Temp\ipykernel_6372\1395517079.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)


Creating constraints...
✅ Constraints created/verified.
Ingesting Knowledge Graph...
✅ Ingested 311 records with full attribute mapping.
Knowledge Graph construction completed successfully.


### POST-INGESTION VERIFICATION

In [142]:
from neo4j import GraphDatabase

def verify_graph():
    driver = GraphDatabase.driver(
        NEO4J_URI,
        auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    queries = {
        # --- Core Entities ---
        "Total Employees": """
            MATCH (e:Employee)
            RETURN count(e) AS cnt
        """,
        "Active Employees": """
            MATCH (e:Employee)
            WHERE e.employment_status = 'Active'
            RETURN count(e) AS cnt
        """,
        "Terminated Employees": """
            MATCH (e:Employee)
            WHERE e.employment_status = 'Terminated'
            RETURN count(e) AS cnt
        """,
        "Departments": """
            MATCH (d:Department)
            RETURN count(d) AS cnt
        """,
        "Positions": """
            MATCH (p:Position)
            RETURN count(p) AS cnt
        """,

        # --- New Insight Nodes ---
        "Locations (States)": """
            MATCH (l:Location)
            RETURN count(l) AS cnt
        """,
        "Performance Categories": """
            MATCH (pr:PerformanceRating)
            RETURN count(pr) AS cnt
        """,
        "Termination Reasons": """
            MATCH (r:Reason)
            RETURN count(r) AS cnt
        """,

        # --- Relationships (Connectivity Check) ---
        "Position History Links (HELD_POSITION)": """
            MATCH ()-[r:HELD_POSITION]->()
            RETURN count(r) AS cnt
        """,
        "Manager Hierarchy Links (REPORTS_TO)": """
            MATCH ()-[r:REPORTS_TO]->()
            RETURN count(r) AS cnt
        """,
        "Location Links (LOCATED_IN)": """
            MATCH ()-[r:LOCATED_IN]->()
            RETURN count(r) AS cnt
        """,
        "Performance Links (HAS_PERFORMANCE_RATING)": """
            MATCH ()-[r:HAS_PERFORMANCE_RATING]->()
            RETURN count(r) AS cnt
        """,
        "Termination Reason Links (TERMINATED_DUE_TO)": """
            MATCH ()-[r:TERMINATED_DUE_TO]->()
            RETURN count(r) AS cnt
        """
    }

    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            print("\n===== GRAPH VERIFICATION RESULTS =====")
            print(f"{'Metric':<45} | {'Count':<10}")
            print("-" * 60)

            for label, cypher in queries.items():
                result = session.run(cypher)
                record = result.single()
                
                if record:
                    cnt = record["cnt"]
                    print(f"{label:<45} | {cnt:<10}")
                else:
                    print(f"{label:<45} | 0 (No Data)")

            # --- Orphan Check (Optional but Recommended) ---
            orphan_query = """
                MATCH (n)
                WHERE NOT (n)--()
                RETURN count(n) AS cnt
            """
            result = session.run(orphan_query)
            orphan_count = result.single()["cnt"]
            print("-" * 60)
            print(f"{'Orphan Nodes (Should be 0)':<45} | {orphan_count:<10}")

    finally:
        driver.close()


# ------------------------------
# Execute Verification
# ------------------------------
verify_graph()


===== GRAPH VERIFICATION RESULTS =====
Metric                                        | Count     
------------------------------------------------------------
Total Employees                               | 334       
Active Employees                              | 207       
Terminated Employees                          | 104       
Departments                                   | 6         
Positions                                     | 32        
Locations (States)                            | 28        
Performance Categories                        | 4         
Termination Reasons                           | 17        
Position History Links (HELD_POSITION)        | 311       
Manager Hierarchy Links (REPORTS_TO)          | 303       
Location Links (LOCATED_IN)                   | 311       
Performance Links (HAS_PERFORMANCE_RATING)    | 311       
Termination Reason Links (TERMINATED_DUE_TO)  | 104       
------------------------------------------------------------
Orphan Nodes

# *NL2Cypher (Natural Language to Cypher):*

### LLM initialisation

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.environ.get("GROQ_API_KEY")

try:
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY not found in environment variables")

    llm = ChatGroq(
        groq_api_key=groq_api_key,
        model_name="llama-instant3.1-8b-",
        temperature=0.7
    )

    # Minimal sanity check call
    llm.invoke("ping")

    print("✅ ChatGroq LLM initialized successfully")

except Exception as e:
    print("❌ ChatGroq LLM initialization failed")
    print(f"Error: {e}")
    raise


✅ ChatGroq LLM initialized successfully


### Canonical KG Schema (Grounded Prompt Context)

In [156]:
KG_SCHEMA = """
You are translating natural language questions into Cypher queries.

====================
GRAPH SCHEMA (STRICT)
====================

Node labels and properties:

(:Employee {
  emp_id: STRING,
  name: STRING,
  salary: FLOAT,
  employment_status: STRING,     
  
  // Dates
  dob: DATE,                     // Format: date('YYYY-MM-DD')
  hire_date: DATE,               
  term_date: DATE,
  last_review: DATE,             // PROPERTY (not a node)

  // Demographics
  sex: STRING,                   
  race: STRING,                  
  marital_status: STRING,        
  citizen_desc: STRING,          

  // Metrics
  engagement_score: FLOAT,
  satisfaction_score: INTEGER,
  absences: INTEGER,
  days_late_last_30: INTEGER,
  project_count: INTEGER
})

(:Department { name: STRING })
(:Source { name: STRING })       
(:Position { title: STRING })
(:Location { state: STRING })    
(:PerformanceRating { rating: STRING }) 
(:Reason { desc: STRING })       

Relationships (STAR SCHEMA - Everything connects to Employee):

(:Employee)-[:WORKS_IN]->(:Department)
(:Employee)-[:RECRUITED_FROM]->(:Source)
(:Employee)-[:HELD_POSITION]->(:Position)
(:Employee)-[:REPORTS_TO]->(:Employee)       
(:Employee)-[:LOCATED_IN]->(:Location)       
(:Employee)-[:HAS_PERFORMANCE_RATING]->(:PerformanceRating)
(:Employee)-[:TERMINATED_DUE_TO]->(:Reason)  

====================
CRITICAL SEMANTIC RULES
====================

1. **NO CHAINING**: Departments, Locations, Sources, and Ratings are NOT connected to each other. They ONLY connect to the Employee.
   - WRONG: `(d:Department)-[:LOCATED_IN]->(l:Location)`
   - RIGHT: `(e:Employee)-[:WORKS_IN]->(d:Department), (e)-[:LOCATED_IN]->(l:Location)`

2. **Properties vs Nodes**: 
   - `last_review` is a DATE PROPERTY on Employee. Do NOT look for a `:LAST_REVIEWED_BY` relationship.
   - `marital_status` is a STRING PROPERTY. Do NOT look for a relationship.

3. **Dates**: ALWAYS use `date('YYYY-MM-DD')`.

4. **Numeric Filters**: 
   - CORRECT: `WHERE e.salary > 50000`
   - WRONG: `{{salary: > 50000}}`

====================
OUTPUT FORMAT
====================

Return ONE Cypher query. 
Nothing else.
"""

2

In [176]:
KG_SCHEMA = """
You are translating natural language questions into Cypher queries.

====================
GRAPH SCHEMA (STRICT)
====================

Node labels and properties:

(:Employee {
  emp_id: STRING,
  name: STRING,          // Format: "Last, First M" (e.g. "Adinolfi, Wilson K")
  salary: FLOAT,
  employment_status: STRING,     
  
  // --- DATE PROPERTIES (NOT RELATIONSHIPS) ---
  dob: DATE,             
  hire_date: DATE,       
  term_date: DATE,       // Used for "when was he terminated"
  last_review: DATE,     // Used for "last review date"

  // --- METRICS ---
  engagement_score: FLOAT,
  satisfaction_score: INTEGER,
  absences: INTEGER,
  days_late_last_30: INTEGER,
  project_count: INTEGER
})

(:Department { name: STRING })
(:Source { name: STRING })       
(:Position { title: STRING })
(:Location { state: STRING })    
(:PerformanceRating { rating: STRING }) 
(:Reason { desc: STRING })       

Relationships (STAR SCHEMA):

(:Employee)-[:WORKS_IN]->(:Department)
(:Employee)-[:RECRUITED_FROM]->(:Source)
(:Employee)-[:HELD_POSITION]->(:Position)
(:Employee)-[:REPORTS_TO]->(:Employee)       
(:Employee)-[:LOCATED_IN]->(:Location)       
(:Employee)-[:HAS_PERFORMANCE_RATING]->(:PerformanceRating)
(:Employee)-[:TERMINATED_DUE_TO]->(:Reason)  // Use ONLY for "WHY", not "WHEN"

====================
CRITICAL RULES
====================

1. **Dates are Properties**: `term_date`, `hire_date`, `last_review` are properties of the `:Employee` node.
   - WRONG: `(e)-[:TERM_DATE]->(...)`
   - RIGHT: `MATCH (e:Employee) WHERE e.term_date > date('...')`

2. **Name Matching**: ALWAYS use `CONTAINS` for names because the format is "Last, First".
   - WRONG: `name = 'Wilson Adinolfi'`
   - RIGHT: `e.name CONTAINS 'Wilson'`

3. **Inequalities**: Put all strict comparisons in the `WHERE` clause.
   - WRONG: `{term_date: > date(...)}`
   - RIGHT: `WHERE e.term_date > date(...)`
"""

3

In [182]:
KG_SCHEMA = """
You are translating natural language questions into Cypher queries.

====================
GRAPH SCHEMA (STRICT)
====================

Node labels and properties:

(:Employee {
  emp_id: STRING,
  name: STRING,          // Format: "Last, First M" (e.g. "Adinolfi, Wilson K")
  salary: FLOAT,
  employment_status: STRING,     
  
  // --- DATE PROPERTIES (NOT RELATIONSHIPS) ---
  dob: DATE,             
  hire_date: DATE,       
  term_date: DATE,       // Used for "when was he terminated"
  last_review: DATE,     // Used for "last review date"

  // --- METRICS ---
  engagement_score: FLOAT,
  satisfaction_score: INTEGER,
  absences: INTEGER,
  days_late_last_30: INTEGER,
  project_count: INTEGER,
  
  // Demographics (Properties)
  sex: STRING,
  race: STRING,
  marital_status: STRING,
  citizen_desc: STRING
})

(:Department { name: STRING })
(:Source { name: STRING })       
(:Position { title: STRING })
(:Location { state: STRING })    
(:PerformanceRating { rating: STRING }) 
(:Reason { desc: STRING })       

Relationships (STAR SCHEMA - Everything connects to Employee):

(:Employee)-[:WORKS_IN]->(:Department)
(:Employee)-[:RECRUITED_FROM]->(:Source)
(:Employee)-[:HELD_POSITION]->(:Position)
(:Employee)-[:REPORTS_TO]->(:Employee)       
(:Employee)-[:LOCATED_IN]->(:Location)       
(:Employee)-[:HAS_PERFORMANCE_RATING]->(:PerformanceRating)
(:Employee)-[:TERMINATED_DUE_TO]->(:Reason)  // Use ONLY for "WHY", not "WHEN"

====================
CRITICAL SEMANTIC RULES
====================

1. **NO CHAINING (Star Schema)**: Departments, Locations, Sources, and Ratings are NOT connected to each other. They ONLY connect to the Employee.
   - WRONG: `(d:Department)-[:LOCATED_IN]->(l:Location)`
   - RIGHT: `(e:Employee)-[:WORKS_IN]->(d:Department), (e)-[:LOCATED_IN]->(l:Location)`

2. **Dates are Properties**: `term_date`, `hire_date`, `last_review` are properties of the `:Employee` node.
   - WRONG: `(e)-[:TERM_DATE]->(...)`
   - RIGHT: `MATCH (e:Employee) WHERE e.term_date > date('...')`

3. **Name Matching**: ALWAYS use `CONTAINS` for names because the format is "Last, First".
   - WRONG: `name = 'Wilson Adinolfi'`
   - RIGHT: `e.name CONTAINS 'Wilson'`

4. **Numeric Inequalities**: Put all strict comparisons in the `WHERE` clause.
   - WRONG: `{{salary: > 50000}}`
   - RIGHT: `WHERE e.salary > 50000`

====================
OUTPUT FORMAT
====================

Return ONE Cypher query. 
Nothing else.
"""

### NL → Cypher Prompt Template

In [171]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""{schema}
You are a Cypher query generator for Neo4j.

====================
FEW-SHOT EXAMPLES (Follow these patterns)
====================

Q: "List all employees in Production department located in MA"
A: MATCH (e:Employee)-[:WORKS_IN]->(d:Department {{name: 'Production'}})
   MATCH (e)-[:LOCATED_IN]->(l:Location {{state: 'MA'}})
   RETURN e.name

Q: "Show employees recruited from LinkedIn with performance Exceeds"
A: MATCH (e:Employee)-[:RECRUITED_FROM]->(s:Source {{name: 'LinkedIn'}})
   MATCH (e)-[:HAS_PERFORMANCE_RATING]->(p:PerformanceRating {{rating: 'Exceeds'}})
   RETURN e.name

Q: "When was the last performance review for Wilson?"
A: MATCH (e:Employee) 
   WHERE e.name CONTAINS 'Wilson'
   RETURN e.last_review

Q: "Employees with engagement score > 4.5 who are Single"
A: MATCH (e:Employee)
   WHERE e.engagement_score > 4.5 AND e.marital_status = 'Single'
   RETURN e.name

====================
YOUR TASK
====================

CRITICAL RULES:
1. Output ONLY a valid Cypher query.
2. Use multiple MATCH clauses for multiple conditions (Star Schema).
3. Do NOT chain relationships that don't exist in the schema.
4. If a query cannot be answered, return: MATCH () WHERE false RETURN null

Question: {question}
Cypher Query:"""
)

cypher_chain = prompt | llm | StrOutputParser()

2

In [177]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""{schema}
You are a Cypher query generator for Neo4j.

====================
FEW-SHOT EXAMPLES (Follow these patterns EXACTLY)
====================

Q: "List employees terminated after 2015-01-01"
A: MATCH (e:Employee)
   WHERE e.term_date > date('2015-01-01')
   RETURN e.name

Q: "Who was hired before 2012?"
A: MATCH (e:Employee)
   WHERE e.hire_date < date('2012-01-01')
   RETURN e.name

Q: "When was the last performance review for Wilson Adinolfi?"
A: MATCH (e:Employee)
   WHERE e.name CONTAINS 'Wilson' AND e.name CONTAINS 'Adinolfi'
   RETURN e.last_review

Q: "Employees with engagement score > 4.5"
A: MATCH (e:Employee)
   WHERE e.engagement_score > 4.5
   RETURN e.name

====================
YOUR TASK
====================

1. Output ONLY a valid Cypher query.
2. **NEVER** use `{{key: > value}}` syntax. ALWAYS use `WHERE key > value`.
3. **NEVER** treat dates as relationships. They are properties.
4. If a query cannot be answered, return: MATCH () WHERE false RETURN null

Question: {question}
Cypher Query:"""
)

cypher_chain = prompt | llm | StrOutputParser()

3

In [183]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""{schema}
You are a Cypher query generator for Neo4j.

====================
FEW-SHOT EXAMPLES (Follow these patterns EXACTLY)
====================

// --- 1. STRUCTURAL QUERIES (Handling relationships) ---

Q: "List all employees in Production department located in MA"
A: MATCH (e:Employee)-[:WORKS_IN]->(d:Department {{name: 'Production'}})
   MATCH (e)-[:LOCATED_IN]->(l:Location {{state: 'MA'}})
   RETURN e.name

Q: "Show employees recruited from LinkedIn with performance Exceeds"
A: MATCH (e:Employee)-[:RECRUITED_FROM]->(s:Source {{name: 'LinkedIn'}})
   MATCH (e)-[:HAS_PERFORMANCE_RATING]->(p:PerformanceRating {{rating: 'Exceeds'}})
   RETURN e.name

// --- 2. TEMPORAL & FILTER QUERIES (Handling syntax) ---

Q: "List employees terminated after 2015-01-01"
A: MATCH (e:Employee)
   WHERE e.term_date > date('2015-01-01')
   RETURN e.name

Q: "Who was hired before 2012?"
A: MATCH (e:Employee)
   WHERE e.hire_date < date('2012-01-01')
   RETURN e.name

Q: "When was the last performance review for Wilson Adinolfi?"
A: MATCH (e:Employee)
   WHERE e.name CONTAINS 'Wilson' AND e.name CONTAINS 'Adinolfi'
   RETURN e.last_review

Q: "Employees with engagement score > 4.5 who are Single"
A: MATCH (e:Employee)
   WHERE e.engagement_score > 4.5 AND e.marital_status = 'Single'
   RETURN e.name

====================
YOUR TASK
====================

1. Output ONLY a valid Cypher query.
2. Use **multiple MATCH clauses** for connecting different entities (Star Schema).
3. **NEVER** chain relationships that don't exist (e.g. Dept -> Location).
4. **NEVER** use `{{key: > value}}` syntax. ALWAYS use `WHERE`.
5. **NEVER** treat dates as relationships. They are properties.

Question: {question}
Cypher Query:"""
)

cypher_chain = prompt | llm | StrOutputParser()

### Generate Cypher from Natural Language

In [184]:
import re

def generate_cypher(nl_query: str) -> str:
    response = cypher_chain.invoke({
        "schema": KG_SCHEMA,
        "question": nl_query
    })

    if not isinstance(response, str):
        raise ValueError("LLM response is not a string")

    response = response.strip()

    # Extract Cypher from code block if present
    code_block = re.search(r"```(?:cypher)?\n(.*?)```", response, re.DOTALL | re.IGNORECASE)

    if code_block:
        cypher = code_block.group(1).strip()
    else:
        cypher = response

    # Hard validation
    cypher_lower = cypher.lower()

    if "match" not in cypher_lower or "return" not in cypher_lower:
        raise ValueError(f"Invalid Cypher generated:\n{response}")

    # Enforce Cypher-only (no prose)
    if ";" in cypher and not cypher.strip().endswith(";"):
        raise ValueError("Multiple statements or malformed Cypher detected")

    return cypher

### Execute Cypher on Neo4j

In [185]:
def run_cypher_query(cypher_query: str):
    driver = GraphDatabase.driver(
        NEO4J_URI,
        auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )

    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            result = session.run(cypher_query)
            records = [record.data() for record in result]
            return records
    finally:
        driver.close()

### End-to-End NL → Cypher → Result

In [186]:
def query_kg(nl_query: str):
    print(f"\nNL Query: {nl_query}")
    
    cypher = generate_cypher(nl_query)
    print("\nGenerated Cypher:")
    print(cypher)
    
    results = run_cypher_query(cypher)
    print("\nResults:")
    for r in results:
        print(r)
    
    return results

### LLM TEST QUERY SET

In [187]:
def run_categorized_tests():
    test_suites = {
        # ---------------------------------------------------------
        # 1. General Queries (Counts, Lists, Basic retrieval)
        # ---------------------------------------------------------
        "General": [
            "How many employees are currently active?",
            "List all the different departments in the company.",
            "What are the different recruitment sources used?",
            "Show me the top 3 highest salaries.",
            # Updated to test new 'citizen_desc' attribute
            "List all employees who are 'US Citizen'." 
        ],

        # ---------------------------------------------------------
        # 2. Temporal Queries (Dates: DOB, Hire, Term, Reviews)
        # ---------------------------------------------------------
        "Temporal": [
            # Tests the new 'hire_date' property on the node
            "Who was hired on 2011-07-05?",
            "Find all employees born before 1980-01-01.",
            "List employees who were terminated after 2015-01-01.",
            "When was the last performance review for employee Wilson Adinolfi?",
            "Find the employee born on 1987-06-14."
        ],

        # ---------------------------------------------------------
        # 3. Hierarchical Queries (Manager -> Report relationships)
        # ---------------------------------------------------------
        "Hierarchical": [
            "Who is the manager of Wilson Adinolfi?",
            "List all employees who report to Michael Albert.",
            "Count how many people report to Janet King.",
            "Does the employee 'Sean Bernstein' have a manager?",
            "Find the manager of the manager of Wilson Adinolfi." 
        ],

        # ---------------------------------------------------------
        # 4. Attribute Constraints (Complex Filtering & New Metrics)
        # ---------------------------------------------------------
        "Attribute Constraints": [
            # Location + Dept
            "List all employees in the 'Production' department who are located in 'MA'.",
            
            # Recruitment + Performance
            "Show me employees recruited from 'LinkedIn' who have a performance rating of 'Exceeds'.",
            
            # New Metric: Absences
            "List all employees who have more than 10 absences.",
            
            # New Metric: Special Projects
            "Who are the employees with a project count greater than 5?",
            
            # Complex: Score + Marital Status (Tests the prompt fix for inequalities)
            "List employees with an engagement score higher than 4.5 who are Single."
        ]
    }

    # Execute Tests
    for category, queries in test_suites.items():
        print(f"\n{'='*40}")
        print(f"🚀 RUNNING: {category.upper()} QUERIES")
        print(f"{'='*40}")
        
        for q in queries:
            try:
                query_kg(q)
            except Exception as e:
                print(f"❌ Error running query '{q}': {e}")

# Run the suite
run_categorized_tests()


🚀 RUNNING: GENERAL QUERIES

NL Query: How many employees are currently active?

Generated Cypher:
MATCH (e:Employee)
WHERE e.employment_status = 'Active'
RETURN COUNT(e)

Results:
{'COUNT(e)': 207}

NL Query: List all the different departments in the company.

Generated Cypher:
MATCH (e:Employee)-[:WORKS_IN]->(d:Department)
RETURN DISTINCT d.name

Results:
{'d.name': 'Production'}
{'d.name': 'IT/IS'}
{'d.name': 'Software Engineering'}
{'d.name': 'Admin Offices'}
{'d.name': 'Sales'}
{'d.name': 'Executive Office'}

NL Query: What are the different recruitment sources used?

Generated Cypher:
MATCH (e:Employee)-[:RECRUITED_FROM]->(s:Source)
RETURN DISTINCT s.name

Results:
{'s.name': 'LinkedIn'}
{'s.name': 'Indeed'}
{'s.name': 'Google Search'}
{'s.name': 'Employee Referral'}
{'s.name': 'Diversity Job Fair'}
{'s.name': 'On-line Web application'}
{'s.name': 'CareerBuilder'}
{'s.name': 'Website'}
{'s.name': 'Other'}

NL Query: Show me the top 3 highest salaries.

Generated Cypher:
MATCH (e:

#  *Direct Cypher query retreival*

In [115]:
direct_test_queries = {
    "GENERAL": [
        # DOB: 06/14/87
        """
        MATCH (e:Employee)
        WHERE e.dob = date('1987-06-14')
        RETURN e.name, e.emp_id, e.dob, e.position_title, e.department
        """,

        #  employee id for salary 140920
        """
        MATCH (e:Employee)
        WHERE e.salary = 140920
        RETURN e.emp_id
        """,

        # Held a position before 2014
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(p:Position)
        WHERE r.from < date('2014-01-01')
        RETURN e.emp_id, e.name, p.title, r.from
        """,

        # Hired between 2010 and 2016
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(:Position)
        WHERE r.from >= date('2010-01-01')
          AND r.from <= date('2016-12-31')
        RETURN e.emp_id, e.name, r.from
        """,

        # Still employed
        """
        MATCH (e:Employee)
        WHERE e.employment_status IS NULL
           OR toLower(e.employment_status) = 'active'
        RETURN e.emp_id, e.name, e.salary
        """
    ],
    "TEMPORAL": [
        # Hired after 2015
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(:Position)
        WHERE r.from > date('2015-12-31')
        RETURN e.emp_id, e.name, r.from
        """,

        # Left after 2018
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(:Position)
        WHERE r.to IS NOT NULL AND r.to > date('2018-12-31')
        RETURN e.emp_id, e.name, r.to
        """,

        # Held a position before 2014
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(p:Position)
        WHERE r.from < date('2014-01-01')
        RETURN e.emp_id, e.name, p.title, r.from
        """,

        # Hired between 2010 and 2016
        """
        MATCH (e:Employee)-[r:HELD_POSITION]->(:Position)
        WHERE r.from >= date('2010-01-01')
          AND r.from <= date('2016-12-31')
        RETURN e.emp_id, e.name, r.from
        """,

        # Still employed
        """
        MATCH (e:Employee)
        WHERE e.employment_status IS NULL
           OR toLower(e.employment_status) = 'active'
        RETURN e.emp_id, e.name, e.salary
        """
    ],

    "HIERARCHICAL": [
        # Reports to manager ID 2
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee {emp_id: '2'})
        RETURN e.emp_id, e.name
        """,

        # Direct reports of employee ID 5
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee {emp_id: '5'})
        RETURN e.emp_id, e.name
        """,

        # Same manager as employee ID 10
        """
        MATCH (e1:Employee {emp_id: '10'})-[:REPORTS_TO]->(m:Employee)
        MATCH (e2:Employee)-[:REPORTS_TO]->(m)
        RETURN DISTINCT e2.emp_id, e2.name
        """,

        # Managers with more than 3 reports
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee)
        WITH m, COUNT(e) AS report_count
        WHERE report_count > 3
        RETURN m.emp_id, m.name, report_count
        """,

        # Employees without manager
        """
        MATCH (e:Employee)
        WHERE NOT (e)-[:REPORTS_TO]->(:Employee)
        RETURN e.emp_id, e.name
        """
    ],

    "ATTRIBUTE": [
        # Salary > 100000
        """
        MATCH (e:Employee)
        WHERE e.salary > 100000
        RETURN e.emp_id, e.salary
        """,

        # Production department
        """
        MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Production'})
        RETURN e.emp_id, e.name
        """,

        # Performance = Exceeds
        """
        MATCH (e:Employee)
        WHERE e.performance_score = 'Exceeds'
        RETURN e.emp_id, e.name
        """,

        # Marital status Single
        """
        MATCH (e:Employee)
        WHERE e.marital_status = 'Single'
        RETURN e.emp_id, e.name
        """,

        # Recruited from Indeed
        """
        MATCH (e:Employee)-[:RECRUITED_FROM]->(s:Source {name: 'Indeed'})
        RETURN e.emp_id, e.name
        """
    ]
}


In [188]:
direct_test_queries = {
    "GENERAL": [
        # 1. How many employees are currently active?
        """
        MATCH (e:Employee)
        WHERE e.employment_status = 'Active'
        RETURN count(e) AS Active_Count
        """,

        # 2. List all the different departments in the company.
        """
        MATCH (d:Department)
        RETURN d.name AS Department
        """,

        # 3. What are the different recruitment sources used?
        """
        MATCH (s:Source)
        RETURN s.name AS Source
        """,

        # 4. Show me the top 3 highest salaries.
        """
        MATCH (e:Employee)
        RETURN e.name, e.salary
        ORDER BY e.salary DESC
        LIMIT 3
        """,

        # 5. List all employees who are 'US Citizen'.
        """
        MATCH (e:Employee)
        WHERE e.citizen_desc = 'US Citizen'
        RETURN e.name, e.emp_id
        """
    ],

    "TEMPORAL": [
        # 1. Who was hired on 2011-07-05?
        """
        MATCH (e:Employee)
        WHERE e.hire_date = date('2011-07-05')
        RETURN e.name, e.hire_date
        """,

        # 2. Find all employees born before 1980-01-01.
        """
        MATCH (e:Employee)
        WHERE e.dob < date('1980-01-01')
        RETURN e.name, e.dob
        """,

        # 3. List employees who were terminated after 2015-01-01.
        """
        MATCH (e:Employee)
        WHERE e.term_date > date('2015-01-01')
        RETURN e.name, e.term_date
        """,

        # 4. When was the last performance review for employee Wilson Adinolfi?
        """
        MATCH (e:Employee)
        WHERE e.name CONTAINS 'Wilson' AND e.name CONTAINS 'Adinolfi'
        RETURN e.name, e.last_review
        """,

        # 5. Find the employee born on 1987-06-14.
        """
        MATCH (e:Employee)
        WHERE e.dob = date('1987-06-14')
        RETURN e.name, e.dob
        """
    ],

    "HIERARCHICAL": [
        # 1. Who is the manager of Wilson Adinolfi?
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee)
        WHERE e.name CONTAINS 'Wilson' AND e.name CONTAINS 'Adinolfi'
        RETURN m.name AS Manager_Name
        """,

        # 2. List all employees who report to Michael Albert.
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee)
        WHERE m.name = 'Michael Albert'
        RETURN e.name AS Direct_Report
        """,

        # 3. Count how many people report to Janet King.
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee)
        WHERE m.name = 'Janet King'
        RETURN count(e) AS Report_Count
        """,

        # 4. Does the employee 'Sean Bernstein' have a manager?
        """
        MATCH (e:Employee {name: 'Sean Bernstein'})
        OPTIONAL MATCH (e)-[:REPORTS_TO]->(m:Employee)
        RETURN e.name, m.name AS Manager
        """,

        # 5. Find the manager of the manager of Wilson Adinolfi.
        """
        MATCH (e:Employee)-[:REPORTS_TO]->(m:Employee)-[:REPORTS_TO]->(big_boss:Employee)
        WHERE e.name CONTAINS 'Wilson' AND e.name CONTAINS 'Adinolfi'
        RETURN big_boss.name AS Manager_of_Manager
        """
    ],

    "ATTRIBUTE": [
        # 1. List all employees in the 'Production' department who are located in 'MA'.
        """
        MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Production'})
        MATCH (e)-[:LOCATED_IN]->(l:Location {state: 'MA'})
        RETURN e.name, d.name, l.state
        """,

        # 2. Show me employees recruited from 'LinkedIn' who have a performance rating of 'Exceeds'.
        """
        MATCH (e:Employee)-[:RECRUITED_FROM]->(s:Source {name: 'LinkedIn'})
        MATCH (e)-[:HAS_PERFORMANCE_RATING]->(p:PerformanceRating {rating: 'Exceeds'})
        RETURN e.name, s.name, p.rating
        """,

        # 3. List all employees who have more than 10 absences.
        """
        MATCH (e:Employee)
        WHERE e.absences > 10
        RETURN e.name, e.absences
        """,

        # 4. Who are the employees with a project count greater than 5?
        """
        MATCH (e:Employee)
        WHERE e.project_count > 5
        RETURN e.name, e.project_count
        """,

        # 5. List employees with an engagement score higher than 4.5 who are Single.
        """
        MATCH (e:Employee)
        WHERE e.engagement_score > 4.5 AND e.marital_status = 'Single'
        RETURN e.name, e.engagement_score, e.marital_status
        """
    ]
}

In [189]:
NEO4J_URI = "neo4j+ssc://b931b8dd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "mW-0f4ShNF394EffXlczpH-5onIZuEsXRFatduQQN5I" 
NEO4J_DATABASE = "neo4j"
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

from neo4j.exceptions import DriverError, ServiceUnavailable

def run_direct_queries():
    try:
        with driver.session(database=NEO4J_DATABASE) as session:
            for category, queries in direct_test_queries.items():
                print(f"\n===== {category} =====")
                for q in queries:
                    result = session.run(q)
                    rows = list(result)
                    print(f"Results: {len(rows)}")
                    for r in rows[:3]:
                        print(dict(r))

    except (DriverError, ServiceUnavailable) as e:
        raise RuntimeError(
            "Neo4j driver is not usable (likely closed). "
            "Recreate the driver and rerun this cell."
        ) from e

run_direct_queries()





===== GENERAL =====
Results: 1
{'Active_Count': 207}
Results: 6
{'Department': 'Production'}
{'Department': 'IT/IS'}
{'Department': 'Software Engineering'}
Results: 9
{'Source': 'LinkedIn'}
{'Source': 'Indeed'}
{'Source': 'Google Search'}
Results: 3
{'e.name': 'Michael Albert', 'e.salary': None}
{'e.name': 'Simon Roup', 'e.salary': None}
{'e.name': 'Kissy Sullivan', 'e.salary': None}
Results: 295
{'e.name': 'Manchester, Robyn', 'e.emp_id': '10077'}
{'e.name': 'Mancuso, Karen', 'e.emp_id': '10073'}
{'e.name': 'Mangal, Debbie', 'e.emp_id': '10279'}

===== TEMPORAL =====
Results: 10
{'e.name': 'Mancuso, Karen', 'e.hire_date': neo4j.time.Date(2011, 7, 5)}
{'e.name': 'Pelletier, Ermine', 'e.hire_date': neo4j.time.Date(2011, 7, 5)}
{'e.name': 'Pham, Hong', 'e.hire_date': neo4j.time.Date(2011, 7, 5)}
Results: 47
{'e.name': 'Manchester, Robyn', 'e.dob': neo4j.time.Date(1976, 8, 25)}
{'e.name': 'Maurice, Shana', 'e.dob': neo4j.time.Date(1977, 11, 22)}
{'e.name': 'Medeiros, Jennifer', 'e.dob': 